In [1]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle

voyages = pd.read_csv("voyages.csv")
X = voyages[['begin_port_id']]
y = voyages[['end_port_id']]

In [2]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

n_estimators = [600,800,1000]
max_depth = [10,15,20]
min_samples_split = [2]
min_samples_leaf = [5] 


hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(RandomForestClassifier(), hyperF, cv = 3, verbose = 1,   n_jobs = -1)
model = gridF.fit(X, y.values.ravel())

Fitting 3 folds for each of 9 candidates, totalling 27 fits


C:\Users\Joakim\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   33.1s finished


In [4]:
print(model.score(X, y))
pickle.dump(model, open('model.pkl', 'wb'))

0.27502921698480715


In [5]:
predict_list = []

for vessel in voyages.drop_duplicates(subset=['vessel'])['vessel'].values:
    current_port = voyages[voyages.vessel == vessel].iloc[-1]['end_port_id']
    prediction_1 = model.predict([[current_port]])[0]
    prediction_2 = model.predict([[prediction_1]])[0]
    prediction_3 = model.predict([[prediction_2]])[0]
    
    predict_list.append({'vessel': vessel, 'begin_port_id': current_port, 'end_port_id': prediction_1, 'voyage': 1})
    predict_list.append({'vessel': vessel, 'begin_port_id': prediction_1, 'end_port_id': prediction_2, 'voyage': 2})
    predict_list.append({'vessel': vessel, 'begin_port_id': prediction_2, 'end_port_id': prediction_3, 'voyage': 3})
    
predict = pd.DataFrame(predict_list)

In [6]:
predict.to_csv('predict.csv', index=False)
